<a href="https://colab.research.google.com/github/Andrian0s/ML4NLP1-2024-Tutorial-Notebooks/blob/main/exercises/ex4/ex4_ner_given_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load and prepare the required data:

In [1]:
chosen_language_code = "de" # German

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import datasets
train_set1000 = datasets.load_dataset('polyglot_ner', chosen_language_code, split='train[:1000]')
train_set3000 = datasets.load_dataset('polyglot_ner', chosen_language_code, split='train[1000:4000]')
test_set = datasets.load_dataset('polyglot_ner', chosen_language_code, split='train[4000:6000]')

print(train_set1000)
print(train_set3000)
print(test_set)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

polyglot_ner.py:   0%|          | 0.00/6.01k [00:00<?, ?B/s]

The repository for polyglot_ner contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/polyglot_ner.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/547578 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'lang', 'words', 'ner'],
    num_rows: 1000
})
Dataset({
    features: ['id', 'lang', 'words', 'ner'],
    num_rows: 3000
})
Dataset({
    features: ['id', 'lang', 'words', 'ner'],
    num_rows: 2000
})


Note: The dataset of polyglot_ner for this language must contain atleast 7000 samples. Please ensure before the language you work with.

### TODO: Inspect and Describe the Data, including Average and Maximum Input length (in tokens)

📝❓Why do you need to be aware of the longest input length within your dataset? Which parameter of the model dictates this?

In [4]:
# since, the labels miss the "B-"" (at the beginning of an entity) and the "I-" (in an entity but not at the beginning) we can prepend those label-parts.
def iob_ify_labels(item):
    """Turn iob-like labels into true IOB format.

    Assumes that all same labels without interruption belong to the same entity.

    Args:
        item: single example of datasets.Dataset
    Returns:
        The modified labels (list of str)
    """
    labels = item['ner']
    iob_labels = []
    previous_label = "O"  # Start with "O" as the previous label

    for label in labels:
        if label == "O":
            iob_labels.append("O")  # No change for "O"
        elif label.startswith("B-") or label.startswith("I-"):
            iob_labels.append(label)  # Keep the existing label if it's already in IOB format
        elif label != previous_label:
            iob_labels.append("B-" + label)  # New entity, add "B-"
        else:
            iob_labels.append("I-" + label)  # Continuation of the same entity, add "I-"

        previous_label = label.split("-")[-1] if "-" in label else label  # Update the previous label
    return {'ner': iob_labels}

# Test case
test_labels = {'ner': ['O', 'PER', 'PER', 'O', 'O', 'O', 'O', 'ORG', 'O', 'O', 'O', 'LOC', 'LOC', 'LOC', 'O']}
print(iob_ify_labels(test_labels)['ner'])


['O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'O']


In [9]:
max_sequence_length = max(
    max(len(item['words']) for item in train_set1000),
    max(len(item['words']) for item in train_set3000),
    max(len(item['words']) for item in test_set)
) #  ? maximum subtoken length. You should use this later

print(max_sequence_length)

154


In [10]:
# map ner-tags to labels
train_set1000 = train_set1000.map(iob_ify_labels)
train_set3000 = train_set3000.map(iob_ify_labels)
test_set = test_set.map(iob_ify_labels)

# Collect all unique NER tags from both datasets
unique_tags_train = set(tag for example in train_set3000['ner'] for tag in example)
unique_tags_test = set(tag for example in test_set['ner'] for tag in example)

# Combine the unique tags from both datasets
all_unique_tags = unique_tags_train.union(unique_tags_test)
print("All Unique NER Tags (IOB Format):", all_unique_tags)

# Create a dictionary to map each unique tag to a numeric label
ner_tag_to_label = {tag: idx for idx, tag in enumerate(sorted(all_unique_tags))}
print("NER Tag to Label Mapping:", ner_tag_to_label)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

All Unique NER Tags (IOB Format): {'I-ORG', 'I-PER', 'B-LOC', 'B-ORG', 'I-LOC', 'O', 'B-PER'}
NER Tag to Label Mapping: {'B-LOC': 0, 'B-ORG': 1, 'B-PER': 2, 'I-LOC': 3, 'I-ORG': 4, 'I-PER': 5, 'O': 6}


### TODO: Map the NER-tags from their string representation to numerical labels.

In [11]:
from transformers import AutoTokenizer
import torch

#load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-german-cased")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/255k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/485k [00:00<?, ?B/s]

In [12]:
def encode_dataset(dataset, tokenizer, max_sequence_length):
    """Encode the input dadaset using a huggingface-tokenizer.

    Args:
        dataset: datasets.Dataset
    Returns:
        The encoded dataset (list of dicts) and lengths (list of ints)
        containing the number of subtokens of each example.
    """
    # Tokenize the dataset
    encoded_dataset = []
    lengths = []

    for item in dataset:
        text = " ".join(item['words']) if isinstance(item['words'], list) else item['words']
        # Tokenize the text
        encoded = tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=max_sequence_length,
            return_tensors='pt'
        )

        # Convert NER tags to numeric labels
        labels = [ner_tag_to_label[tag] for tag in item['ner']]
        labels += [0] * (max_sequence_length - len(labels))  # Pad labels with 0s

        encoded_dataset.append({
            'input_ids': encoded['input_ids'].squeeze(),
            'attention_mask': encoded['attention_mask'].squeeze(),
            'token_type_ids': encoded.get('token_type_ids', torch.tensor([0] * max_sequence_length)).squeeze(),
            'labels': torch.tensor(labels, dtype=torch.long)
        })
        lengths.append(len(encoded['input_ids'].squeeze()))

    return encoded_dataset, lengths

In [13]:
unique_tags = set(tag for item in train_set1000 for tag in item['ner'])
print(unique_tags)


{'I-ORG', 'I-PER', 'B-LOC', 'B-ORG', 'I-LOC', 'O', 'B-PER'}


In [14]:
# encode the two training sets and the test set
enc_trainset1000, train_lengths1000 = encode_dataset(train_set1000, tokenizer, max_sequence_length)
enc_trainset3000, train_lengths3000 = encode_dataset(train_set3000, tokenizer, max_sequence_length)
enc_testset, test_lengths = encode_dataset(test_set, tokenizer, max_sequence_length)

In [15]:
# check out how the training sets are encoded
for key, val in enc_trainset1000[0].items():
    print(f'{key}: {val.size()}')

input_ids: torch.Size([154])
attention_mask: torch.Size([154])
token_type_ids: torch.Size([154])
labels: torch.Size([154])


Example of how your output could look like.

input_ids: torch.Size([???])

token_type_ids: torch.Size([???])

attention_mask: torch.Size([???])

labels: torch.Size([???])

📝❓What value should replace the three question marks in your print? Should this be the same for all samples? Why/Why not?

In [16]:
# from transformers import What are the three imports from transformers we need? Hint: Look at the tutorial notebook.
import os
os.environ["WANDB_MODE"] = "disabled"

### TODO: Initialise your model and set up your training arguments

📝❓When initializing the BertForTokenClassification-class with BERT-base you should get a warning message. Explain why you get this message.


In [17]:
from transformers import BertForTokenClassification, BertTokenizer, TrainingArguments, Trainer

# Initialize the tokenizer and model for German
tokenizer = BertTokenizer.from_pretrained("bert-base-german-cased")
model = BertForTokenClassification.from_pretrained(
    "bert-base-german-cased",
    num_labels=len(ner_tag_to_label)  # Number of unique labels in your dataset
)

# Set up the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,  # Adjust based on your GPU memory
    per_device_eval_batch_size=16,   # Adjust based on your GPU memory
    num_train_epochs=4,              # Number of epochs
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10
)


model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-german-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### TODO Train your Model ⚡ GPU 2-3 mins

Suggested: 4 Epochs with training batch size of 16/32/64 - Depends on the available GPU memory after loading the model

In [18]:
from transformers import DataCollatorWithPadding

# Update the Trainer to use the data collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=enc_trainset1000,
    eval_dataset=enc_testset,
)

# Start training
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss
1,0.058700,0.056289
2,0.036800,0.048621
3,0.036800,0.048222
4,0.025400,0.048423


TrainOutput(global_step=252, training_loss=0.06795083772805002, metrics={'train_runtime': 172.6777, 'train_samples_per_second': 23.165, 'train_steps_per_second': 1.459, 'total_flos': 314386871568000.0, 'train_loss': 0.06795083772805002, 'epoch': 4.0})

### TODO: Compute Metrics/Performance (Macro and Micro F1) of your model.

📝❓ Is there a challenge when evaluating the predictions of your model? Why is this challenge present and how do you plan to deal with it?

Hint: Look at the predictions lengths and distributions of labels

To avoid rerunning, please also print the metrics of each model that completed training

In [19]:
from sklearn.metrics import f1_score

# Use the Trainer to get predictions on the evaluation dataset
outputs = trainer.predict(enc_testset)

# Extract the logits (raw predictions) and labels
logits = outputs.predictions
labels = outputs.label_ids

# Convert logits to predicted labels
predictions = logits.argmax(axis=-1)

def compute_metrics(predictions, labels):
    # Flatten the predictions and labels to remove padding tokens
    true_labels = []
    pred_labels = []

    for pred, label in zip(predictions, labels):
        for p, l in zip(pred, label):
            if l != 0:  # Ignore padding tokens (assuming 0 is the padding index)
                true_labels.append(l)
                pred_labels.append(p)

    # Calculate Macro and Micro F1 scores
    macro_f1 = f1_score(true_labels, pred_labels, average='macro')
    micro_f1 = f1_score(true_labels, pred_labels, average='micro')

    return {"macro_f1": macro_f1, "micro_f1": micro_f1}

# Example usage
metrics = compute_metrics(predictions, labels)
print(metrics)

# Delete the trainer to free up memory
del trainer
torch.cuda.empty_cache()  # Clear GPU memory if using a GPU

{'macro_f1': 0.22291655931681667, 'micro_f1': 0.9241737245843671}


### TODO: Prepare and run the other three variants described ⚡ 2 - 10 minutes each.

In [20]:
# Whole Embeddings, 3000 samples:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=enc_trainset3000,
    eval_dataset=enc_testset
)
trainer.train()

# Use the Trainer to get predictions on the evaluation dataset
outputs = trainer.predict(enc_testset)

# Extract the logits (raw predictions) and labels
logits = outputs.predictions
labels = outputs.label_ids

# Convert logits to predicted labels
predictions = logits.argmax(axis=-1)

metrics = compute_metrics(predictions, labels)
print(metrics)

# Delete the trainer to free up memory
del trainer
torch.cuda.empty_cache()  # Clear GPU memory if using a GPU

Epoch,Training Loss,Validation Loss
1,0.038800,0.038263
2,0.023100,0.033455
3,0.018300,0.033930
4,0.013900,0.034649


{'macro_f1': 0.3593862281683933, 'micro_f1': 0.9368922348646876}


In [21]:
# Freeze the embedding layer
for param in model.bert.embeddings.parameters():
    param.requires_grad = False

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=enc_trainset1000,
    eval_dataset=enc_testset
)
trainer.train()

# Use the Trainer to get predictions on the evaluation dataset
outputs = trainer.predict(enc_testset)

# Extract the logits (raw predictions) and labels
logits = outputs.predictions
labels = outputs.label_ids

# Convert logits to predicted labels
predictions = logits.argmax(axis=-1)

metrics = compute_metrics(predictions, labels)
print(metrics)

# Delete the trainer to free up memory
del trainer
torch.cuda.empty_cache()  # Clear GPU memory if using a GPU

Epoch,Training Loss,Validation Loss
1,0.027500,0.039646
2,0.012900,0.039854
3,0.014800,0.040270
4,0.008500,0.042292


{'macro_f1': 0.3296302053709918, 'micro_f1': 0.9325861921464624}


In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=enc_trainset3000,
    eval_dataset=enc_testset
)
trainer.train()

# Use the Trainer to get predictions on the evaluation dataset
outputs = trainer.predict(enc_testset)

# Extract the logits (raw predictions) and labels
logits = outputs.predictions
labels = outputs.label_ids

# Convert logits to predicted labels
predictions = logits.argmax(axis=-1)

metrics = compute_metrics(predictions, labels)
print(metrics)

# Delete the trainer to free up memory
del trainer
torch.cuda.empty_cache()  # Clear GPU memory if using a GPU

Epoch,Training Loss,Validation Loss
1,0.016100,0.038131
2,0.008300,0.042833
3,0.005600,0.045522
4,0.004000,0.047858


{'macro_f1': 0.3549333122413021, 'micro_f1': 0.9318162374882368}


📝❓Template:

Summary of Performance of the four Model Variants


1. Whole Model finetuning, 1000 samples:
2. Frozen Embeddings, 1000 samples:
3. Whole Model finetuning, 3000 samples:
3. Frozen Embeddings 3000 samples:




📝❓Which model performed best on the evaluation set?




📝❓How large is the performance gap between 1,000 and 3,000 sentences for finetuning?


📝❓ Are there differences between f1-micro and f1-macro score? If so, why?

📝❓ Is it better to freeze or not to freeze the embeddings? Hypothesize why

## Report: Named Entity Recognition Task Analysis

### 1. Why do you need to be aware of the longest input length within your dataset? Which parameter of the model dictates this?
The length of input is crucial for the efficiency of the model. If the input length is too long, the model's performance and efficiency can be negatively impacted. The `max_sequence_length` parameter dictates the maximum allowable length of input sequences. This parameter ensures that sequences are either truncated or padded to maintain uniformity, which is essential for efficient processing.

### 2. What value should replace the three question marks in your print? Should this be the same for all samples? Why/Why not?
The value that should replace the question marks is `154`, which corresponds to `max_sequence_length`(which was obtained by finding the length of the longest sentence from all 3 datasets). This length should be the same for all samples to optimize the efficiency of the model. When processing inputs of uniform size, the model can parallelize operations more effectively. If input lengths were inconsistent, the model would require additional resources to handle the variations, reducing overall efficiency.

### 3. Is there a challenge when evaluating the predictions of your model? Why is this challenge present and how do you plan to deal with it?
Yes, the challenge lies in the presence of padding tokens. Padding tokens do not contribute to meaningful predictions but are necessary to maintain uniform sequence lengths. Including them in evaluations would distort the model's performance metrics. To address this, we use an attention mask, which tells the model which tokens are actual words and which are padding. This ensures the model ignores padding tokens during evaluation, preserving the validity of the performance metrics.

### 4. Performance Metrics Table
| Training Set       | Embedding Type    | Macro F1 Score | Micro F1 Score |
|--------------------|-------------------|----------------|----------------|
| Trainset 1000      | Whole Embedding   | 0.2229         | 0.9246         |
| Trainset 3000      | Whole Embedding   | 0.3593         | 0.9368         |
| Trainset 1000      | Frozen Embedding  | 0.3296         | 0.9325         |
| Trainset 3000      | Frozen Embedding  | 0.3549         | 0.9318         |

### 5. Which model performed best on the evaluation set?
The model trained on `trainset_3000` with whole embedding performed the best, achieving the highest Macro and Micro F1 scores.

### 6. How large is the performance gap between 1,000 and 3,000 sentences for fine-tuning?
The performance gap in Micro F1 score between 1,000 and 3,000 sentences is relatively small. However, in terms of Macro F1 score, there is a notable difference. The gap is especially significant for models with whole embedding, highlighting the impact of using more data on performance across all classes.

### 7. Are there differences between F1-micro and F1-macro scores? If so, why?
Yes, there are differences. The F1-micro score gives more weight to frequent classes, while the F1-macro score treats all classes equally. Our dataset is highly imbalanced, with the `O` class being much more frequent than the named entity classes. As a result, the F1-macro score is lower, reflecting poorer performance on less frequent classes.

### 8. Is it better to freeze or not to freeze the embeddings? Hypothesize why.
When the dataset is small, such as `trainset_1000`, it is better to freeze the embeddings. This approach outperforms whole embedding in both Macro and Micro F1 scores, likely because freezing prevents overfitting. However, as the dataset size increases, whole embedding becomes more effective. With more data (`trainset_3000`), whole embedding outperforms frozen embedding, suggesting that the model can better adapt to the specific task with more data for fine-tuning. This difference is likely due to the ability of whole embedding to leverage the larger dataset to learn task-specific features.
